In [1]:
import argparse
import collections
import json
import logging
import os
import sys

from komorebi.libs.model.attention_configuration import AttentionConfiguration
from komorebi.libs.model.attention_model import AttentionModel 
from komorebi.libs.dataset.dataset_config import DatasetConfiguration
from komorebi.libs.model.parameter_initialization import ParameterInitializationPolicy
from komorebi.libs.optimizer.optimizer_config import OptimizerConfiguration
from komorebi.libs.optimizer.optimizer_factory import create_tf_optimizer 
from komorebi.libs.trainer.attention_trainer import AttentionTrainer
from komorebi.libs.trainer.trainer_config import TrainerConfiguration
from komorebi.libs.utilities.io_utils import copy_data, ensure_directory, load_pickle_object, remove_directory

In [ ]:
# Tensorflow specific directories
CHECKPOINT_DIRECTORY_NAME = "model_checkpoints"
CONFIG_DIRECTORY_NAME = "config"
SUMMARY_DIRECTORY_NAME = "training_summaries"

ExperimentConfiguration = collections.namedtuple(typename='ExperimentConfiguration', 
                                                 field_names=['experiment_directory',
                                                              'checkpoint_directory',
                                                              'config_directory',
                                                              'summary_directory',
                                                              'dataset_config', 
                                                              'model_config', 
                                                              'trainer_config', 
                                                              'optimizer_config', 
                                                              'parameter_initialization'])

def _parse_experiment_config_json(experiment_config_json):
    """Parse experiment json and convert to experiment config.

    Parse relevant fields in experiment configuration file. 

    :param config_json: path to experiment config json.
    :return: 
        ExperimentConfiguration named-tuple with the following attributes.
            experiment_name      : name of experiment
            experiment_directory : base directory to create experiment directory 
            dataset_config              : dataset configuration object
            model_config                : model configuration object
            trainer_config              : trainer configuration object
            optimizer_config            : optimizer configuration object
    """
    with open(experiment_config_json, 'r') as f:
        experiment_info = json.load(f)

        experiment_directory = os.path.join(experiment_info['experiments_directory'], 
                                            experiment_info['experiment_name'])
        checkpoint_directory = os.path.join(experiment_directory, CHECKPOINT_DIRECTORY_NAME)
        config_directory = os.path.join(experiment_directory, CONFIG_DIRECTORY_NAME)
        summary_directory = os.path.join(experiment_directory, SUMMARY_DIRECTORY_NAME)
        
        return ExperimentConfiguration(experiment_directory=experiment_directory,
                                       checkpoint_directory=checkpoint_directory,
                                       config_directory=config_directory,
                                       summary_directory=summary_directory,
                                       parameter_initialization=ParameterInitializationPolicy(),
                                       dataset_config=_create_dataset_configuration(experiment_info['dataset_config']),
                                       model_config=_create_model_configuration(experiment_info['model_config']),
                                       trainer_config=_create_trainer_configuration(experiment_info['trainer_config']),
                                       optimizer_config=_create_optimizer_configuration(experiment_info['trainer_config']))


In [2]:
dataset_path = "/Users/andy/Projects/biology/research/komorebi/data/attention_validation_dataset/sharded_attention_dataset.pkl"
dataset = load_pickle_object(dataset_path)

In [3]:
import time

dataset_times = []
for _ in range(10):
    
    t0 = time.time()
    dataset.get_training_examples(range(8000))
    t1 = time.time()
    dataset_times.append(t1-t0)

In [5]:
sum(dataset_times)/len(dataset_times)

17.861739897727965

## Simple Profiling Example

In [6]:
import tensorflow as tf
from tensorflow.python.client import timeline

a = tf.random_normal([2000, 5000])
b = tf.random_normal([5000, 1000])
res = tf.matmul(a, b)

with tf.Session() as sess:
    # add additional options to trace the session execution
    options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()
    sess.run(res, options=options, run_metadata=run_metadata)

    # Create the Timeline object, and write it to a json file
    fetched_timeline = timeline.Timeline(run_metadata.step_stats)
    chrome_trace = fetched_timeline.generate_chrome_trace_format()
    with open('timeline_01.json', 'w') as f:
        f.write(chrome_trace)